In [1]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
pth = '../Log-files/'

In [2]:
pths = ['ai-kd.csv', 'base-kd.csv', 'no-distil.csv', 'reverseKL-KD.csv', 'teacher-KD.csv']
data = []

for pth1 in pths:
    df = pd.read_csv(pth + pth1)
    data.append(df['model_answer'])

In [3]:
nli_model = AutoModelForSequenceClassification.from_pretrained('roberta-large-mnli').to(device)
nli_tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli')

model.safetensors:  97%|#########7| 1.38G/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/anaconda3/envs/torch_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
def check_logical_consistency(reasoning):
    steps = reasoning.split('. ')
    for i in range(len(steps) - 1):
        premise = steps[i]
        hypothesis = steps[i + 1]
        inputs = nli_tokenizer.encode(premise, hypothesis, return_tensors='pt', truncation=True).to(device)
        with torch.no_grad():
            logits = nli_model(inputs)[0]
        # Index 0: Contradiction, 1: Neutral, 2: Entailment
        entailment_score = torch.softmax(logits, dim=1)[0][2].item()
        if entailment_score < 0.5:
            return False
    return True


In [ ]:
for i, set in enumerate(data):
    logical_consistency = [check_logical_consistency(ans) for ans in set]
    consistency_accuracy = sum(logical_consistency) / len(logical_consistency)
    with open('output.txt', 'a') as f:
        f.write(f'Consistency Accuracy for {pths[i]}: {consistency_accuracy}\n')